In [1]:
import pyvips  #must use conda to install
import os
import glob
from natsort import natsorted
from time import time

In [2]:
src = '/Volumes/Digital pathology image lib/JHU/Laura Wood/IF for SenPan001/SerialSecAlign/DalignIm/indsf5_outdsf1'
sec2 = [_ for _ in os.listdir(src) if 'sec002' in _]
sec5 = [_ for _ in os.listdir(src) if 'sec005' in _]

In [15]:
secs = sec2+sec5
secs

['WSI_sec002_round01_ch1.png',
 'WSI_sec002_round01_ch2.png',
 'WSI_sec002_round01_ch3.png',
 'WSI_sec002_round01_ch4.png',
 'WSI_sec002_round02_ch1.png',
 'WSI_sec002_round02_ch2.png',
 'WSI_sec002_round02_ch3.png',
 'WSI_sec002_round02_ch4.png',
 'WSI_sec005_round01_ch1.png',
 'WSI_sec005_round01_ch2.png',
 'WSI_sec005_round01_ch3.png',
 'WSI_sec005_round01_ch4.png',
 'WSI_sec005_round02_ch1.png',
 'WSI_sec005_round02_ch2.png',
 'WSI_sec005_round02_ch3.png',
 'WSI_sec005_round02_ch4.png']

In [3]:
sec2s = []
for idx,im in enumerate(sec2):
    impth = os.path.join(src, im)
    imobj = pyvips.Image.new_from_file(impth)
    # imobj = pyvips.Image.openslideload(impth,level=0)
    if imobj.hasalpha(): imobj = imobj[:-1]
    sec2s.append(imobj)

In [4]:
sec5s = []
for idx,im in enumerate(sec2):
    impth = os.path.join(src, im)
    imobj = pyvips.Image.new_from_file(impth)
    # imobj = pyvips.Image.openslideload(impth,level=0)
    if imobj.hasalpha(): imobj = imobj[:-1]
    sec5s.append(imobj)

In [16]:
sec25 = []
for idx,im in enumerate(secs):
    impth = os.path.join(src, im)
    imobj = pyvips.Image.new_from_file(impth)
    # imobj = pyvips.Image.openslideload(impth,level=0)
    if imobj.hasalpha(): imobj = imobj[:-1]
    sec25.append(imobj)

In [5]:
image_height = imobj.height
image_width = imobj.width

In [6]:
comp2 = pyvips.Image.arrayjoin(sec2s, across=1)
comp5 = pyvips.Image.arrayjoin(sec5s, across=1)

In [17]:
comp25 = pyvips.Image.arrayjoin(sec25, across=1)

In [18]:
comp25 = comp25.copy()

In [7]:
comps = pyvips.Image.arrayjoin([comp2,comp5], across=1)

In [8]:
comp2 = comp2.copy()
comp5 = comp5.copy()
comps = comps.copy()

In [9]:
comp2.set_type(pyvips.GValue.gint_type, "page-height", image_height)
comp2.set_type(pyvips.GValue.gstr_type, "image-description",
              f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Image ID="Image:0">
        <!-- Minimum required fields about image dimensions -->
        <Pixels DimensionOrder="XYCZT"
                ID="Pixels:0"
                SizeC="8"
                SizeT="1"
                SizeX="{image_width}"
                SizeY="{image_height}"
                SizeZ="1"
                Type="uint8">
        </Pixels>
    </Image>
</OME>""")

In [10]:
start=time()
comp2.tiffsave('comp2.ome.tiff', compression="jpeg", tile=True,
              tile_width=512, tile_height=512,
              pyramid=True, subifd=True)
print('elapsed time: {}'.format(round(time()-start)))

elapsed time: 8


In [19]:
comp25.set_type(pyvips.GValue.gint_type, "page-height", image_height)
comp25.set_type(pyvips.GValue.gstr_type, "image-description",
              f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Image ID="Image:0">
        <!-- Minimum required fields about image dimensions -->
        <Pixels DimensionOrder="XYCZT"
                ID="Pixels:0"
                SizeC="8"
                SizeT="1"
                SizeX="{image_width}"
                SizeY="{image_height}"
                SizeZ="2"
                Type="uint8">
        </Pixels>
    </Image>
</OME>""")

In [20]:
start=time()
comp25.tiffsave('comp25.ome.tiff', compression="jpeg", tile=True,
              tile_width=512, tile_height=512,
              pyramid=True, subifd=True)
print('elapsed time: {}'.format(round(time()-start)))

elapsed time: 16


In [12]:
def png2multiCmultiZOME(ims,outputfilepath):
    imobjs = []
    for idx,im in enumerate(ims):
        impth = os.path.join(input_dir, im)
        imobj = pyvips.Image.new_from_file(impth)
        # imobj = pyvips.Image.openslideload(impth,level=0)
        if imobj.hasalpha(): imobj = imobj[:-1]
        imobjs.append(imobj)
        # print('image {} has dimensions [{},{}]'.format(idx,imobj.height,imobj.width))
    # print('z stack height :',len(imobjs))
    comp = pyvips.Image.arrayjoin(imobjs, across=1)
    image_height = comp.height
    image_width = comp.width
    comp = comp.copy()

    # set minimal OME metadata
    # before we can modify an image (set metadata in this case), we must take a
    # private copy
    comp.set_type(pyvips.GValue.gint_type, "page-height", image_height)
    comp.set_type(pyvips.GValue.gstr_type, "image-description",
                  f"""<?xml version="1.0" encoding="UTF-8"?>
    <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
        xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
        <Image ID="Image:0">
            <!-- Minimum required fields about image dimensions -->
            <Pixels DimensionOrder="XYCZT"
                    ID="Pixels:0"
                    SizeC="8"
                    SizeT="1"
                    SizeX="{image_width}"
                    SizeY="{image_height}"
                    SizeZ="42"
                    Type="uint8">
            </Pixels>
        </Image>
    </OME>""")


    #jpeg,jp2k,lzw,
    print('saving image...')
    # jp2k breaks the format somehow
    start=time()
    comp.tiffsave(outputfilepath, compression="none", tile=True,
                  tile_width=512, tile_height=512,
                  pyramid=True, subifd=True)
    print('elapsed time: {}'.format(round(time()-start)))
    print('ome-tiff saved here: ',outputfilepath)

In [ ]:
if __name__=='__main__':
    input_dir = '/Volumes/Digital pathology image lib/JHU/Laura Wood/IF for SenPan001/SerialSecAlign/DalignIm/indsf5_outdsf1'
    # output_dir = os.path.join(input_dir, 'cstack')
    # if not os.path.exists(output_dir): os.mkdir(output_dir)
    ims = glob.glob(os.path.join(input_dir,'*.png'))
    ims = natsorted(ims)
    print(len(ims))
    png2multiCmultiZOME(ims,os.path.join(input_dir,'c8z42.ome.tiff'))

336
saving image...
